In [38]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset, DatasetDict
from transformers import pipeline # Προστέθηκε για το pipeline
import os
import shutil # Προστέθηκε για το shutil.rmtree

# 1. Ρυθμίσεις
model_name = "nlpaueb/bert-base-greek-uncased-v1"
# Δεν ορίζουμε output_dir για αποθήκευση, καθώς δεν θα αποθηκεύσουμε
text_file_path = r'C:\Users\alexi\OneDrive\Υπολογιστής\NLP\BONUS\source_code\training_data.txt'

# 2. Φόρτωση Tokenizer & Μοντέλου
print(f"Φορτώνει τον tokenizer και το μοντέλο '{model_name}'...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)
print("Tokenizer και μοντέλο φορτώθηκαν επιτυχώς.")

# 3. Φόρτωση Dataset
try:
    print(f"\nΦορτώνει δεδομένα από το αρχείο: {text_file_path}")
    with open(text_file_path, 'r', encoding='utf-8') as f:
        # Διαβάζει γραμμές και αφαιρεί κενές, κάθε γραμμή είναι ένα παράδειγμα
        lines = [line.strip() for line in f if line.strip()] 
    
    raw_dataset = Dataset.from_dict({"text": lines})
    print(f"Το περιεχόμενο του αρχείου φορτώθηκε επιτυχώς. Αριθμός παραδειγμάτων: {len(raw_dataset)}")

    # **ΣΗΜΑΝΤΙΚΟ:** Διαίρεση του dataset σε train και test splits
    # Διατηρείται ο περιορισμός στα 1000 παραδείγματα όπως τον είχες
    small_dataset = raw_dataset.select(range(min(4000, len(raw_dataset)))) 
    train_test_split_dataset = small_dataset.train_test_split(test_size=0.1) 
    print(f"Το dataset διαιρέθηκε: Train set: {len(train_test_split_dataset['train'])} παραδείγματα, Test set: {len(train_test_split_dataset['test'])} παραδείγματα.")

except FileNotFoundError:
    print(f"Σφάλμα: Το αρχείο '{text_file_path}' δεν βρέθηκε. Βεβαιωθείτε ότι το αρχείο υπάρχει και η διαδρομή είναι σωστή.")
    exit() 
except Exception as e:
    print(f"Προέκυψε ένα απροσδόκητο σφάλμα: {e}")
    exit() 

# 4. Tokenization
# Διατηρείται το max_length=64 όπως το είχες
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=64)

print("\nΞεκινάει το Tokenization του dataset...")
# Εφαρμογή της λειτουργίας tokenization στο DatasetDict (train και test splits)
tokenized_dataset = train_test_split_dataset.map(tokenize_function, batched=True, remove_columns=["text"])
print("Το Tokenization ολοκληρώθηκε.")

# 5. Data collator για [MASK] tokens
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

# 6. Ορισμός Training Arguments
# Διατηρείται ο προσωρινός φάκελος και οι ρυθμίσεις για μη αποθήκευση όπως τις είχες
temp_output_dir = "./temp_trainer_output" # Προσωρινός φάκελος για λογαριασμό του Trainer
if not os.path.exists(temp_output_dir):
    os.makedirs(temp_output_dir)

training_args = TrainingArguments(
    output_dir=temp_output_dir,           # Προσωρινός κατάλογος για logs/internal checkpoints (δεν αποθηκεύουμε το τελικό)
    overwrite_output_dir=True,            # Αντικαθιστά τα υπάρχοντα αποτελέσματα στον temp_output_dir
    num_train_epochs=1,                   # Αριθμός εποχών εκπαίδευσης (διατηρείται η 1 εποχή)
    per_device_train_batch_size=16,       # Μέγεθος batch ανά συσκευή (GPU/CPU)
    save_steps=-1,                        # Απενεργοποιεί την αποθήκευση checkpoints
    save_total_limit=0,                   # Δεν διατηρεί καθόλου checkpoints
    prediction_loss_only=True,            
    report_to="none",                     # Απενεργοποιεί την αναφορά σε external services (π.χ. W&B)
)

# 7. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"], 
    data_collator=data_collator,
)

# 8. Εκκίνηση fine-tuning
print("\nΞεκινάει το fine-tuning του μοντέλου (χωρίς αποθήκευση στο δίσκο)...")
trainer.train()
print("Το fine-tuning ολοκληρώθηκε.")

# 9. Δεν υπάρχει βήμα αποθήκευσης του μοντέλου/tokenizer εδώ.

# 10. Δοκιμή του fine-tuned μοντέλου
print("\n--- Δοκιμή του μοντέλου (που βρίσκεται στη μνήμη) ---")
# Χρησιμοποιούμε απ' ευθείας το trainer.model που είναι το εκπαιδευμένο μοντέλο στη μνήμη.
fill_mask_pipeline = pipeline("fill-mask", model=trainer.model, tokenizer=tokenizer)

# Παράδειγμα πρόβλεψης που έχεις δώσει


Φορτώνει τον tokenizer και το μοντέλο 'nlpaueb/bert-base-greek-uncased-v1'...
Tokenizer και μοντέλο φορτώθηκαν επιτυχώς.

Φορτώνει δεδομένα από το αρχείο: C:\Users\alexi\OneDrive\Υπολογιστής\NLP\BONUS\source_code\training_data.txt
Το περιεχόμενο του αρχείου φορτώθηκε επιτυχώς. Αριθμός παραδειγμάτων: 7088
Το dataset διαιρέθηκε: Train set: 3600 παραδείγματα, Test set: 400 παραδείγματα.

Ξεκινάει το Tokenization του dataset...


Map: 100%|██████████| 400/400 [00:00<00:00, 9003.36 examples/s]


Το Tokenization ολοκληρώθηκε.

Ξεκινάει το fine-tuning του μοντέλου (χωρίς αποθήκευση στο δίσκο)...


c:\Users\alexi\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


Device set to use cpu


Το fine-tuning ολοκληρώθηκε.

--- Δοκιμή του μοντέλου (που βρίσκεται στη μνήμη) ---


In [39]:
# --- ΝΕΑ ΣΥΝΑΡΤΗΣΗ ΓΙΑ ΣΥΜΠΛΗΡΩΣΗ ΠΑΡΑΓΡΑΦΟΥ ---
import unicodedata
def remove_greek_accents(text):
    """
    Αφαιρεί τους ελληνικούς τόνους από μια συμβολοσειρά.
    """
    # Κανονικοποιεί τη συμβολοσειρά σε μορφή NFD (Normalization Form Canonical Decomposition)
    # όπου οι τόνοι διαχωρίζονται από τα γράμματα.
    # Στη συνέχεια, φιλτράρει τους χαρακτήρες που είναι "mark" (όπως οι τόνοι).
    return ''.join(c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn')


# --- ΝΕΑ ΣΥΝΑΡΤΗΣΗ ΓΙΑ ΣΥΜΠΛΗΡΩΣΗ ΠΑΡΑΓΡΑΦΟΥ ---
def fill_and_display_paragraph(text_with_masks: str, fm_pipeline) -> tuple[str, list[str]]:
    """
    Συμπληρώνει ένα κείμενο με [MASK] tokens χρησιμοποιώντας το fill-mask pipeline,
    εκτυπώνει τις καλύτερες προβλέψεις και επιστρέφει την συμπληρωμένη παράγραφο
    και τη λίστα με τις προβλεφθείσες λέξεις.

    Args:
        text_with_masks (str): Το κείμενο που περιέχει τα [MASK] tokens.
        fm_pipeline: Το Hugging Face fill-mask pipeline.

    Returns:
        tuple[str, list[str]]: Η παράγραφος με όλα τα [MASK] tokens συμπληρωμένα
                                και μια λίστα με τις προβλεφθείσες λέξεις (στη σειρά).
    """
    print(f"Αρχικό κείμενο για συμπλήρωση:")
    print(f"'{text_with_masks}'")
    print("-" * 60)

    predictions_for_text = fm_pipeline(text_with_masks, top_k=1)

    predicted_words = [] 

    for mask_idx, mask_predictions in enumerate(predictions_for_text):
        if mask_predictions:
            best_prediction = mask_predictions[0] 
            predicted_word = best_prediction['token_str']
            predicted_words.append(predicted_word) 

            print(f"--- Καλύτερη πρόβλεψη για το [MASK] {mask_idx + 1} ---")
            print(f"'{best_prediction['sequence']}' (Πιθανότητα: {best_prediction['score']:.4f}, Λέξη: '{predicted_word}')")
        print("-" * 60)

    completed_paragraph = text_with_masks
    for word in predicted_words:
        completed_paragraph = re.sub(r'\[MASK\]', word, completed_paragraph, 1)

    print("\n--- Συμπληρωμένη Παράγραφος από το Μοντέλο ---")
    print(completed_paragraph)
    print("-" * 60)

    return completed_paragraph, predicted_words 

# --- ΣΥΝΑΡΤΗΣΗ ΓΙΑ ΣΥΓΚΡΙΣΗ ΚΕΙΜΕΝΩΝ (difflib) ---
def compare_texts(text1: str, text2: str, label1="Κείμενο 1", label2="Κείμενο 2"):
    print(f"\n--- Σύγκριση: {label1} vs {label2} (Αναλυτικό) ---")
    
    # Εφαρμόζουμε remove_greek_accents πριν τον καθαρισμό και τη σύγκριση
    cleaned_text1 = " ".join(remove_greek_accents(text1).split()).replace(" .", ".").replace(" ,", ",").replace(" —", "—")
    cleaned_text2 = " ".join(remove_greek_accents(text2).split()).replace(" .", ".").replace(" ,", ",").replace(" —", "—")

    differ = difflib.Differ()
    diff = list(differ.compare(cleaned_text1.splitlines(), cleaned_text2.splitlines()))

    for line in diff:
        print(line)

    print("\nΟδηγίες σύγκρισης (από το difflib):")
    print("  ' ' : γραμμή ίδια")
    print("  '-' : γραμμή που υπάρχει μόνο στο πρώτο κείμενο")
    print("  '+' : γραμμή που υπάρχει μόνο στο δεύτερο κείμενο")
    print("  '?' : δείχνει τις διαφορές χαρακτήρων μέσα σε μια γραμμή (με ^)")
    print("-" * 60)

# --- ΠΑΡΑΔΕΙΓΜΑ ΧΡΗΣΗΣ ---

# Το κείμενο με τα MASK tokens που θα δώσουμε στο μοντέλο
text_to_fill = """Πραγματική δουλεία σε [MASK] η υπέρ του κοινού ακινήτου. — Στο κοινό 
[MASK] μπορεί να συσταθεί πραγματική δουλεία υπέρ του [MASK] κύριου άλλου ακινήτου 
και αν ακόμη αυτός είναι [MASK] του ακινήτου που βαρύνεται με τη δουλεία. Το ίδιο ισχύει 
και για την [MASK] δουλεία πάνω σε ακίνητο υπέρ των εκάστοτε κυρίων κοινού ακινήτου, 
αν [MASK] από αυτούς είναι κύριος του [MASK] που βαρύνεται με τη δουλεία. """

# Το ground truth κείμενο για σύγκριση
ground_truth_text = """ Πραγματική  Δουλεία υπέρ κοινού ακινήτου. - Στο κοινό ακίνητο μπορεί να συσταθεί πραγματική δουλεία υπέρ του
 εκάστοτε κυρίου άλλου ακινήτου και αν ακόμη αυτός είναι συγκύριος του
 ακινήτου που βαρύνεται με τη δουλεία. Το ίδιο ισχύει και για πραγματική
 δουλεία πάνω σε ακίνητο υπέρ των εκάστοτε κυρίων κοινού ακινήτου, αν
 κάποιος από αυτούς είναι κύριος του ακινήτου που βαρύνεται με τη
 δουλεία."""

# **Οι σωστές λέξεις που αντιστοιχούν σε κάθε [MASK] με τη σειρά.**
# Τώρα, μπορούμε να τις γράψουμε και με τόνους, καθώς η συνάρτηση αξιολόγησης θα τους αφαιρέσει
correct_masked_words = ["ακίνητο", "ακίνητο", "εκάστοτε", "συγκύριος", "πραγματική", "κάποιος", "ακινήτου"]

# 1. Συμπληρώνουμε την παράγραφο χρησιμοποιώντας τη συνάρτηση
predicted_paragraph, predicted_words_list = fill_and_display_paragraph(text_to_fill, fill_mask_pipeline)

# 2. Συγκρίνουμε την συμπληρωμένη παράγραφο με το ground truth (αναλυτικά)
compare_texts(ground_truth_text, predicted_paragraph, 
              label1="Ground Truth", label2="Πρόβλεψη Μοντέλου")

# 3. Ποσοτική Αξιολόγηση των προβλεφθέντων λέξεων
print("\n--- Ποσοτική Αξιολόγηση Λέξεων [MASK] ---")

if len(predicted_words_list) != len(correct_masked_words):
    print("Προσοχή: Ο αριθμός των προβλεφθέντων λέξεων δεν ταιριάζει με τον αριθμό των σωστών λέξεων.")
    print(f"Προβλεφθείσες: {len(predicted_words_list)}, Σωστές: {len(correct_masked_words)}")
else:
    correct_predictions_count = 0
    total_masks = len(correct_masked_words)

    print(f"Συγκρίνοντας {total_masks} MASKed θέσεις:")
    for i in range(total_masks):
        # Εφαρμόζουμε remove_greek_accents και lower() και στις δύο λέξεις
        pred_word_normalized = remove_greek_accents(predicted_words_list[i]).strip().lower()
        corr_word_normalized = remove_greek_accents(correct_masked_words[i]).strip().lower()

        if pred_word_normalized == corr_word_normalized:
            correct_predictions_count += 1
            status = "✔ Σωστό"
        else:
            status = "✖ Λάθος"
        print(f"  MASK {i+1}: Πρόβλεψη='{predicted_words_list[i]}' (κανονικοποιημένο: '{pred_word_normalized}') | Σωστό='{correct_masked_words[i]}' (κανονικοποιημένο: '{corr_word_normalized}') -> {status}")

    accuracy = (correct_predictions_count / total_masks) * 100
    print(f"\nΑποτελέσματα:")
    print(f"  Συνολικές προβλέψεις MASK: {total_masks}")
    print(f"  Σωστές προβλέψεις MASK: {correct_predictions_count}")
    print(f"  Ακρίβεια (masked words accuracy): {accuracy:.2f}%")

print("-" * 60)


Αρχικό κείμενο για συμπλήρωση:
'Πραγματική δουλεία σε [MASK] η υπέρ του κοινού ακινήτου. — Στο κοινό 
[MASK] μπορεί να συσταθεί πραγματική δουλεία υπέρ του [MASK] κύριου άλλου ακινήτου 
και αν ακόμη αυτός είναι [MASK] του ακινήτου που βαρύνεται με τη δουλεία. Το ίδιο ισχύει 
και για την [MASK] δουλεία πάνω σε ακίνητο υπέρ των εκάστοτε κυρίων κοινού ακινήτου, 
αν [MASK] από αυτούς είναι κύριος του [MASK] που βαρύνεται με τη δουλεία. '
------------------------------------------------------------
--- Καλύτερη πρόβλεψη για το [MASK] 1 ---
'[CLS] πραγματικη δουλεια σε ακινητο η υπερ του κοινου ακινητου. — στο κοινο [MASK] μπορει να συσταθει πραγματικη δουλεια υπερ του [MASK] κυριου αλλου ακινητου και αν ακομη αυτος ειναι [MASK] του ακινητου που βαρυνεται με τη δουλεια. το ιδιο ισχυει και για την [MASK] δουλεια πανω σε ακινητο υπερ των εκαστοτε κυριων κοινου ακινητου, αν [MASK] απο αυτους ειναι κυριος του [MASK] που βαρυνεται με τη δουλεια. [SEP]' (Πιθανότητα: 0.4677, Λέξη: 'ακινητο')
------

In [40]:
# Το κείμενο με τα MASK tokens που θα δώσουμε στο μοντέλο
text_to_fill = """Κοινό πράγμα. — Αν η κυριότητα του [MASK]  ανήκει σε περισσότερους 
[MASK] αδιαιρέτου κατ΄ιδανικά [MASK], εφαρμόζονται οι διατάξεις για την κοινωνία. """

# Το ground truth κείμενο για σύγκριση
ground_truth_text = """ Κοινό πράγμα. — Αν η κυριότητα του πράγματος  ανήκει σε περισσότερους 
εξ αδιαιρέτου κατ΄ιδανικά μέρη, εφαρμόζονται οι διατάξεις για την κοινωνία."""

# **Οι σωστές λέξεις που αντιστοιχούν σε κάθε [MASK] με τη σειρά.**
# Τώρα, μπορούμε να τις γράψουμε και με τόνους, καθώς η συνάρτηση αξιολόγησης θα τους αφαιρέσει
correct_masked_words = ["πράγματος", "εξ", "μέρη"]

# 1. Συμπληρώνουμε την παράγραφο χρησιμοποιώντας τη συνάρτηση
predicted_paragraph, predicted_words_list = fill_and_display_paragraph(text_to_fill, fill_mask_pipeline)

# 2. Συγκρίνουμε την συμπληρωμένη παράγραφο με το ground truth (αναλυτικά)
compare_texts(ground_truth_text, predicted_paragraph, 
              label1="Ground Truth", label2="Πρόβλεψη Μοντέλου")

# 3. Ποσοτική Αξιολόγηση των προβλεφθέντων λέξεων
print("\n--- Ποσοτική Αξιολόγηση Λέξεων [MASK] ---")

if len(predicted_words_list) != len(correct_masked_words):
    print("Προσοχή: Ο αριθμός των προβλεφθέντων λέξεων δεν ταιριάζει με τον αριθμό των σωστών λέξεων.")
    print(f"Προβλεφθείσες: {len(predicted_words_list)}, Σωστές: {len(correct_masked_words)}")
else:
    correct_predictions_count = 0
    total_masks = len(correct_masked_words)

    print(f"Συγκρίνοντας {total_masks} MASKed θέσεις:")
    for i in range(total_masks):
        # Εφαρμόζουμε remove_greek_accents και lower() και στις δύο λέξεις
        pred_word_normalized = remove_greek_accents(predicted_words_list[i]).strip().lower()
        corr_word_normalized = remove_greek_accents(correct_masked_words[i]).strip().lower()

        if pred_word_normalized == corr_word_normalized:
            correct_predictions_count += 1
            status = "✔ Σωστό"
        else:
            status = "✖ Λάθος"
        print(f"  MASK {i+1}: Πρόβλεψη='{predicted_words_list[i]}' (κανονικοποιημένο: '{pred_word_normalized}') | Σωστό='{correct_masked_words[i]}' (κανονικοποιημένο: '{corr_word_normalized}') -> {status}")

    accuracy = (correct_predictions_count / total_masks) * 100
    print(f"\nΑποτελέσματα:")
    print(f"  Συνολικές προβλέψεις MASK: {total_masks}")
    print(f"  Σωστές προβλέψεις MASK: {correct_predictions_count}")
    print(f"  Ακρίβεια (masked words accuracy): {accuracy:.2f}%")

print("-" * 60)

Αρχικό κείμενο για συμπλήρωση:
'Κοινό πράγμα. — Αν η κυριότητα του [MASK]  ανήκει σε περισσότερους 
[MASK] αδιαιρέτου κατ΄ιδανικά [MASK], εφαρμόζονται οι διατάξεις για την κοινωνία. '
------------------------------------------------------------
--- Καλύτερη πρόβλεψη για το [MASK] 1 ---
'[CLS] κοινο πραγμα. — αν η κυριοτητα του κοινου ανηκει σε περισσοτερους [MASK] αδιαιρετου κατ΄ιδανικα [MASK], εφαρμοζονται οι διαταξεις για την κοινωνια. [SEP]' (Πιθανότητα: 0.3748, Λέξη: 'κοινου')
------------------------------------------------------------
--- Καλύτερη πρόβλεψη για το [MASK] 2 ---
'[CLS] κοινο πραγμα. — αν η κυριοτητα του [MASK] ανηκει σε περισσοτερους του αδιαιρετου κατ΄ιδανικα [MASK], εφαρμοζονται οι διαταξεις για την κοινωνια. [SEP]' (Πιθανότητα: 0.5763, Λέξη: 'του')
------------------------------------------------------------
--- Καλύτερη πρόβλεψη για το [MASK] 3 ---
'[CLS] κοινο πραγμα. — αν η κυριοτητα του [MASK] ανηκει σε περισσοτερους [MASK] αδιαιρετου κατ΄ιδανικα μερη, εφαρμο